<a href="https://colab.research.google.com/github/PeterPirog/tf-autoencoders/blob/main/01_mnist_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#VAE CODER
import tensorflow.keras as keras

from keras import layers
from keras import backend as K 
from keras.models import Model
import numpy as np

img_shape=(28,28,1)
batch_size=16
latent_dim =2

input_img=keras.Input(shape=img_shape)

x=layers.Conv2D(32,3,padding='same',activation='relu')(input_img)
x=layers.Cov2D(64,3,padding='same',activation='relu',strides=(2,2))(x)
